<a href="https://colab.research.google.com/github/mkoko22/AMP_Generation_using_LDM/blob/main/generation/Phase_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import wandb
from google.colab import drive

drive.mount('/content/drive')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

diff_config = {
    "latent_dim": 64,
    "n_steps": 500,
    "lr": 0.0001,
    "batch_size": 512,
    "epochs": 100,
    "gamma": 0.99,
    "device": device
}


def get_paper_params(n_steps, device):
    betas = torch.linspace(0.0001, 0.02, n_steps).to(device)
    alphas = 1.0 - betas
    alphas_cumprod = torch.cumprod(alphas, dim=0)
    return betas, alphas, alphas_cumprod

betas, alphas, alphas_cumprod = get_paper_params(diff_config["n_steps"], device)


class BERTDiffusion(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        # Three fully connected networks
        self.time_mlp = nn.Embedding(diff_config["n_steps"], 512)
        self.input_proj = nn.Linear(latent_dim, 512)
        self.output_proj = nn.Linear(512, latent_dim)

        # Embedding layers
        self.pos_embed = nn.Parameter(torch.randn(1, 1, 512))

        # BERT encoding layer
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=512,
            nhead=8,
            dim_feedforward=2048,
            dropout=0.1,
            batch_first=True
        )
        self.bert_encoding_layer = nn.TransformerEncoder(encoder_layers, num_layers=4)

        # Normalization and Dropout
        self.layer_norm = nn.LayerNorm(512)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x, t):
        # FC Processing
        x_feat = self.input_proj(x).unsqueeze(1)
        t_emb = self.time_mlp(t).unsqueeze(1)

        # Feature Addition: Input + Time + Position
        h = x_feat + t_emb + self.pos_embed

        # Normalization and Dropout
        h = self.layer_norm(h)
        h = self.dropout(h)

        # BERT Backbone
        h = self.bert_encoding_layer(h)

        # Final FC Processing
        return self.output_proj(h.squeeze(1))

train_latents = torch.load("/content/drive/MyDrive/AMP-Generation/data/latent_uncond_train.pth")
val_latents = torch.load("/content/drive/MyDrive/AMP-Generation/data/latent_uncond_val.pth")

train_loader = DataLoader(TensorDataset(train_latents), batch_size=diff_config["batch_size"], shuffle=True)
val_loader = DataLoader(TensorDataset(val_latents), batch_size=diff_config["batch_size"])

model = BERTDiffusion(diff_config["latent_dim"]).to(device)
optimizer = optim.AdamW(model.parameters(), lr=diff_config["lr"])
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=diff_config["gamma"])
criterion = nn.MSELoss()

checkpoint_path = None
start_epoch = 0

if checkpoint_path is not None and os.path.exists(checkpoint_path):
    print(f"--- Loading Checkpoint: {checkpoint_path} ---")
    ckpt = torch.load(checkpoint_path, map_location=device)
    if isinstance(ckpt, dict) and 'model_state_dict' in ckpt:
        model.load_state_dict(ckpt['model_state_dict'])
        optimizer.load_state_dict(ckpt['optimizer_state_dict'])
        start_epoch = ckpt['epoch']
    else:
        model.load_state_dict(ckpt)
        start_epoch = 50
    print(f"Resuming at Epoch {start_epoch}")

wandb.init(project="AMP-Latent-Diffusion-Full-Paper", config=diff_config, resume="allow")


@torch.no_grad()
def generate_samples(model, num_samples=512):
    model.eval()
    xt = torch.randn(num_samples, diff_config["latent_dim"]).to(device)

    for t_idx in reversed(range(diff_config["n_steps"])):
        t_batch = torch.full((num_samples,), t_idx, device=device, dtype=torch.long)

        x0_pred = model(xt, t_batch)

        if t_idx > 0:
            a_bar = alphas_cumprod[t_idx]
            a_bar_prev = alphas_cumprod[t_idx - 1]
            beta_t = betas[t_idx]
            alpha_t = alphas[t_idx]
            mu = ((torch.sqrt(a_bar_prev) * beta_t) / (1 - a_bar)) * x0_pred + \
                 ((torch.sqrt(alpha_t) * (1 - a_bar_prev)) / (1 - a_bar)) * xt

            var = ((1 - a_bar_prev) / (1 - a_bar)) * beta_t
            xt = mu + torch.sqrt(var) * torch.randn_like(xt)
        else:
            xt = x0_pred

    return xt


for epoch in range(start_epoch, diff_config["epochs"]):
    model.train()
    train_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        x0 = batch[0].to(device)
        t = torch.randint(0, diff_config["n_steps"], (x0.shape[0],), device=device)

        epsilon = torch.randn_like(x0)
        a_bar = alphas_cumprod[t].view(-1, 1)
        xt = torch.sqrt(a_bar) * x0 + torch.sqrt(1 - a_bar) * epsilon

        x0_pred = model(xt, t)
        loss = criterion(x0_pred, x0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    scheduler.step()


    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            x0_v = batch[0].to(device)
            t_v = torch.randint(0, diff_config["n_steps"], (x0_v.shape[0],), device=device)
            a_bar_v = alphas_cumprod[t_v].view(-1, 1)
            xt_v = torch.sqrt(a_bar_v) * x0_v + torch.sqrt(1 - a_bar_v) * torch.randn_like(x0_v)
            v_pred = model(xt_v, t_v)
            val_loss += criterion(v_pred, x0_v).item()

    avg_train = train_loss / len(train_loader)
    avg_val = val_loss / len(val_loader)

    wandb.log({
        "train_loss": avg_train,
        "val_loss": avg_val,
        "learning_rate": scheduler.get_last_lr()[0],
        "epoch": epoch + 1
    })

    print(f"Epoch {epoch+1} | Loss: {avg_train:.6f} | Val: {avg_val:.6f} | LR: {scheduler.get_last_lr()[0]:.6f}")


    if (epoch + 1) % 5 == 0:
        save_path = f"/content/drive/MyDrive/AMP-Generation/checkpoints/diffusion_paper_final_ep{epoch+1}.pth"
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_val,
        }, save_path)

wandb.finish()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


...

Epoch 99: 100%|██████████| 892/892 [00:30<00:00, 29.00it/s]


Epoch 99 | Loss: 1.212304 | Val: 1.187907 | LR: 0.000037


Epoch 100: 100%|██████████| 892/892 [00:30<00:00, 28.84it/s]


Epoch 100 | Loss: 1.211141 | Val: 1.188702 | LR: 0.000037


epoch,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█████
learning_rate,█████▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁
train_loss,█▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▅▄▃▃▃▃▄▃▃▃▂▃▂▃▂▂▂▂▃▂▂▃▃▂▁▃▂▂▃▂▃▁▃▂▂▂▂▂
epoch,100
learning_rate,4e-05
train_loss,1.21114
val_loss,1.1887
